In [18]:
from fullmethod import display_usecases

In [19]:
display_usecases()

,Description,Dataset Prefix
0,What factors contribute to bus revenues?,demo_lake/busridertbl.csv
1,What factors contribute to soccer players' success?,demo_lake/soccerplayertbl.csv
2,What factors affect hospital congestion?,demo_lake/hospitaltbl.csv
3,What factors affect bank revenues?,demo_lake/banktbl.csv
4,Does where a politician was educated affect their popularity?,demo_lake/politiciantbl.csv
5,What other factors help me predict churn for banking customers?,demo_lake/Churn_Modelling.csv
6,What other factors would help me predict audit risk?,demo_lake/audit_risk.csv
7,What factors can help me determine if a client subscribe a term deposit?,demo_lake/bank_full.csv
8,What factors help in predicting credit card default?,demo_lake/default.csv
9,How can I segment bank customers?,demo_lake/german_credit.csv


In [20]:
from numex_apis import show_lshjoins

In [21]:
show_lshjoins('demo_lake/busridertbl.csv')

,Output Table,Output Table Join Key,Input Table Join Key
0,demo_lake/politicianjoin.csv,Unnamed: 0,Unnamed: 0
1,demo_lake/soccertbl.csv,Unnamed: 0.1,Unnamed: 0
2,demo_lake/Team_Attributes.csv,chanceCreationPassing,Unnamed: 0
3,demo_lake/Team_Attributes.csv,buildUpPlayPassing,Unnamed: 0
4,demo_lake/soccertbl.csv,Unnamed: 0,Unnamed: 0
...,...,...,...
56,demo_lake/busridertbl.csv,dbo:BusCompany,dbo:BusCompany
57,demo_lake/busridertbl.csv,<http://dbpedia.org/property/annualRidership>,<http://dbpedia.org/property/annualRidership>
58,demo_lake/busridertbl.csv,<http://dbpedia.org/ontology/numberOfLines>,<http://dbpedia.org/ontology/numberOfLines>
59,demo_lake/busridertbl.csv,dbo:regionServed,dbo:regionServed


In [22]:
from numex_apis import show_semantic_labels
show_semantic_labels('demo_lake/busridertbl.csv')

,Column Name,Best Semantic Label,Membership Probability
0,<http://dbpedia.org/property/annualRidership>,"(['dbo:BusCompany'], '<http://dbpedia.org/property/annualRidership>')",0.99999
1,<http://dbpedia.org/ontology/numberOfLines>,"(['dbo:BusCompany'], '<http://dbpedia.org/ontology/numberOfLines>')",0.99999


In [23]:
from numex_apis import avg_member_entropy

avg_member_entropy('demo_lake/busridertbl.csv', 
                   'demo_lake/busriderjoin.csv')

,Average Entropy,Recommended Method,Entropy Threshold
0,0.328888,KG-based,0.5


In [24]:
import pandas as pd
from numex_apis import disambiguate_onerow

indf = pd.read_csv('demo_lake/busridertbl.csv')
outdf = pd.read_csv('demo_lake/busriderjoin.csv')
joindf = indf.merge(outdf, left_on='dbo:regionServed', right_on='dbo:regionServed')
row0 = joindf.to_dict(orient='records')[0]

disambiguate_onerow('demo_lake/busridertbl.csv', 'demo_lake/busriderjoin.csv', row0)

({('dbo:BusCompany',): ['http://dbpedia.org/resource/Shuttle–UM']},
 {(): ['http://dbpedia.org/resource/Montgomery_County,_Maryland']})

In [25]:
from numex_apis import find_relationships

find_relationships({('dbo:BusCompany',): ['http://dbpedia.org/resource/Shuttle–UM']},
                  {(): ['http://dbpedia.org/resource/Montgomery_County,_Maryland']})

[(('dbo:BusCompany',), 'http://dbpedia.org/ontology/regionServed', ()),
 (('dbo:BusCompany',), 'http://dbpedia.org/property/serviceArea', ())]

In [26]:
from numex_apis import find_kgstrength
find_kgstrength('demo_lake/busridertbl.csv', 'demo_lake/busriderjoin.csv')

(0.91,
 {(('dbo:BusCompany',), 'http://dbpedia.org/property/serviceArea', ()): 184,
  (('dbo:BusCompany',), 'http://dbpedia.org/ontology/regionServed', ()): 184},
 202)

In [27]:
from numex_apis import show_kgmatch
show_kgmatch('demo_lake/busridertbl.csv', 'demo_lake/busriderjoin.csv')

Input Table: demo_lake/busridertbl.csv 
 
 
   
 Unnamed: 0 
 BusCompany 
 annualRidership 
 numberOfLines 
 regionServed 
 regionServed_id 
 
 
 
 
 0 
 0 
 Shuttle–UM 
 2956600.00 
 31 
 Montgomery_County,_Maryland 
 2 
 
 
 1 
 1 
 Razorback_Transit 
 1989087.00 
 19 
 Washington_County,_Arkansas 
 3 
 
 
 2 
 2 
 El_Metro_Transit 
 4300000.00 
 24 
 Webb_County,_Texas 
 4 
 
 
 3 
 3 
 Pace_(transit) 
 28392400.00 
 218 
 Will_County,_Illinois 
 5 
 
 
 4 
 4 
 Roaring_Fork_Transportation_Authority 
 5470000.00 
 3 
 Eagle_County,_Colorado 
 1 
 
 
 

 
 KG Join Table: demo_lake/busriderjoin.csv
Relationship Strength: 0.91 
 
 
   
 Unnamed: 0 
 regionServed 
 areaTotal 
 percentageOfAreaWater 
 regionServed_id 
 
 
 
 
 0 
 0 
 Montgomery_County,_Maryland 
 1313.12 
 0.03 
 2 
 
 
 1 
 1 
 Washington_County,_Arkansas 
 2464.94 
 0.60 
 3 
 
 
 2 
 2 
 Webb_County,_Texas 
 8741.21 
 0.40 
 4 
 
 
 3 
 3 
 Will_County,_Illinois 
 2198.90 
 0.01 
 5 
 
 
 4 
 4 
 Eagle_County,_Colorado 
 4382.26 
 0.40 
 1

In [36]:
from numex_apis import show_topk_kgmatches
show_topk_kgmatches('demo_lake/busridertbl.csv', 3)

,Rank,Table Name,Join Key,Score,Best Relationship
0,1,demo_lake/busriderjoin.csv,dbo:regionServed,0.91,http://dbpedia.org/ontology/regionServed


In [37]:
from numex_apis import find_proxy_table
find_proxy_table('demo_lake/ETF prices.csv')

,Proxy Table,Normalized Numeric Feature Difference
0,demo_lake/indexProcessed.csv,0.112529


In [38]:
show_lshjoins('demo_lake/indexProcessed.csv')

,Output Table,Output Table Join Key,Input Table Join Key
0,demo_lake/indexProcessed.csv,Index,Index
1,demo_lake/MutualFund prices - F-K.csv,price_date,Date
2,demo_lake/MutualFund prices - A-E.csv,price_date,Date
3,demo_lake/indexProcessed.csv,Date,Date
4,demo_lake/ETF prices.csv,price_date,Date
5,demo_lake/indexProcessed.csv,Open,Open
6,demo_lake/indexProcessed.csv,Adj Close,High
7,demo_lake/indexProcessed.csv,Close,High
8,demo_lake/indexProcessed.csv,High,High
9,demo_lake/indexProcessed.csv,Low,Low


In [39]:
from numex_apis import find_diststrength
find_diststrength('demo_lake/ETF prices.csv', 'demo_lake/MutualFund prices - A-E.csv')

0.00010831546535126091

In [42]:
from numex_apis import show_distmatch
show_distmatch('demo_lake/ETF prices.csv', 'demo_lake/MutualFund prices - A-E.csv')

Input Table: demo_lake/ETF prices.csv 
 
 
   
 fund_symbol 
 price_date 
 open 
 high 
 low 
 close 
 adj_close 
 volume 
 
 
 
 
 0 
 AAA 
 2020-09-09 
 25.10 
 25.12 
 25.07 
 25.07 
 24.85 
 17300 
 
 
 1 
 AAA 
 2020-09-10 
 25.06 
 25.07 
 25.05 
 25.07 
 24.85 
 23500 
 
 
 2 
 AAA 
 2020-09-11 
 25.04 
 25.05 
 25.02 
 25.03 
 24.81 
 33400 
 
 
 3 
 AAA 
 2020-09-14 
 25.01 
 25.06 
 25.01 
 25.02 
 24.80 
 13100 
 
 
 4 
 AAA 
 2020-09-15 
 25.02 
 25.03 
 25.01 
 25.01 
 24.79 
 12100 
 
 
 

 
 Non-KG Proxy Table: demo_lake/indexProcessed.csv 
 
 
   
 Index 
 Date 
 Open 
 High 
 Low 
 Close 
 Adj Close 
 Volume 
 CloseUSD 
 
 
 
 
 0 
 HSI 
 1986-12-31 
 2568.30 
 2568.30 
 2568.30 
 2568.30 
 2568.30 
 0.00 
 333.88 
 
 
 1 
 HSI 
 1987-01-02 
 2540.10 
 2540.10 
 2540.10 
 2540.10 
 2540.10 
 0.00 
 330.21 
 
 
 2 
 HSI 
 1987-01-05 
 2552.40 
 2552.40 
 2552.40 
 2552.40 
 2552.40 
 0.00 
 331.81 
 
 
 3 
 HSI 
 1987-01-06 
 2583.90 
 2583.90 
 2583.90 
 2583.90 
 2583.90 
 0.00 
 335.91 
 
 
 4 
 HSI 
 1987-01-07 
 2607.10 
 2607.10 
 2607.10 
 2607.10 
 2607.10 
 0.00 
 338.92 
 
 
 

 
 Non-KG Join Table: demo_lake/MutualFund prices - A-E.csv
Relationship Strength: 0.0001083 
 
 
   
 fund_symbol 
 price_date 
 nav_per_share 
 
 
 
 
 0 
 AAAAX 
 2007-07-31 
 10.02 
 
 
 1 
 AAAAX 
 2007-08-01 
 9.98 
 
 
 2 
 AAAAX 
 2007-08-02 
 10.01 
 
 
 3 
 AAAAX 
 2007-08-03 
 9.90 
 
 
 4 
 AAAAX 
 2007-08-06 
 9.93

In [40]:
from numex_apis import show_topk_distmatches
show_topk_distmatches('demo_lake/ETF prices.csv', 3)

,Input Table,Proxy Table,Output Table,Distribution-based Strength
0,demo_lake/ETF prices.csv,demo_lake/indexProcessed.csv,demo_lake/MutualFund prices - A-E.csv,0.000108
1,demo_lake/ETF prices.csv,demo_lake/indexProcessed.csv,demo_lake/MutualFund prices - F-K.csv,0.000107
